In [12]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [13]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 15.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [14]:
path_dict = {
    'train': 'data/Соответствия.xlsx',
    'etalon': 'data/Эталонная номенклатура.xlsx',
}


In [15]:
train = pd.read_excel(path_dict['train'])


In [16]:
etalon = pd.read_excel(path_dict['etalon'])


In [17]:
N = 1000
train = train.iloc[:N]
etalon = etalon.iloc[:N]


In [18]:
etalon

,Номенклатура
0,AS17-L100x50x10-30-CS3
1,"Hardtop XP Std Comp A RAL 6011 Jotun 17,33л"
2,"Hardtop XP Std Comp B Jotun 1,8л"
3,"Hardtop XP Std Comp B RAL 1023 Jotun 1,8л"
4,"Hardtop XP Std Comp B RAL 6011 Jotun 1,8л."
...,...
995,"Бобышка БП1-М20х1,5-100 ст.10Х17Н13М2Т ТУ 36-1..."
996,"Бобышка БП1-М20х1,5-100 ст.13ХФА ТУ 4218-032-0..."
997,"Бобышка БП1-М20х1,5-100 ст.20 ТУ 36-1097-85"
998,"Бобышка БП1-М20х1,5-100 ст.20 УХЛ ТУ 4218-032-..."


In [19]:
# !pip install name_matching


In [20]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

etalon_column_name = 'Номенклатура'



# initialise the name matcher
matcher = NameMatcher(number_of_matches=1,
                      legal_suffixes=True,
                      common_words=False,
                      top_n=50,
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column=etalon_column_name,
                                     df_matching_data=etalon,
                                     transform=True)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=train,
                              column_matching='Описание')



preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 32.21it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 1000/1000 [01:28<00:00, 11.36it/s]

done


In [25]:
combined = pd.merge(train, matches, how='left', left_index=True, right_on='match_index')
selected_columns = ['Описание', 'Эталонная номенклатура', 'score']
combined = combined[selected_columns]



In [28]:
combined.sort_values(by='score', ascending=False).head(1000)


,Описание,Эталонная номенклатура,score
855.0,06 1082755880 d22 d113 201 n6 261,Блок пружинный 06 ОСТ 108.275.58-80,100.000000
466.0,07 ct 341074393,Блок пружинный 07 ОСТ 34-10-743-93,100.000000
464.0,07 ct 341074393,Блок пружинный 07 ОСТ 34-10-743-93,100.000000
862.0,hst3 12115 4020,Анкер-шпилька М12х115 HILTI HST,100.000000
242.0,06 1082755880 d22 d113 201 n6 261,Блок пружинный 06 ОСТ 108.275.58-80,100.000000
...,...,...,...
915.0,1640 779870 14,Болт М16х40 ГОСТ 7798-70 оцинк.,16.147680
575.0,1596 092 1423872011,Втулка ВСЭ-А-159х6 ст.09Г2С ТУ 14-2Р-387-2011,15.648538
696.0,hilti h s a m12100,Анкер-шпилька М12х110 HILTI HSA,15.614504
697.0,630 69 341072693,Блок подвески 630У-69 ОСТ 34-10-726-93,15.303857
